In [1]:
import re
import os
from glob import glob
from bs4 import BeautifulSoup
import random
import json

In [2]:
# Get list of prose fiction HTML files

In [3]:
htmlFilenames = open('pg-prose-fic-html-files.txt').read().split()

In [4]:
"/run/media/jon/SAMSUNG/gitenberg/The-Legenduof-Sleepy-Hollow_41/41-h/41-h.htm" in htmlFilenames

True

In [5]:
htmlFilenames[8]

'/run/media/jon/SAMSUNG/gitenberg/December-Love_6616/6616-h/6616-h.htm'

In [6]:
class Text():
    def __init__(self, filename):
        print('Using file: ', filename)
        self.filename = filename
        self.rawText = open(filename, errors="ignore").read()
        self.soup = BeautifulSoup(self.rawText, 'html.parser')
        self.text = self.soup.get_text()
        self.length = len(self.text)
        self.matches = self.findNamedAnchors(filename)
#         print(self.matches)
        self.naAreChapters = [self.naIsChapter(match) for match in self.matches]
        
        self.numChapters = 0
        self.chapLocs = []
        if True in self.naAreChapters: 
            self.numChapters = self.naAreChapters.count(True)
            for i, m in enumerate(self.matches): 
                if self.naAreChapters[i]: # Is it a chapter heading? 
                    loc = m.span()[0]
                    self.chapLocs.append(loc)
        else: 
            self.h3matches = self.findH3()
#             print(self.h3matches)
            self.numChapters = len(self.h3matches)
        
        self.chapLocs.append(self.getEndLocation())
        
        self.chaps = self.getTextBetweenHeadings()
        self.chapObjs = [Chapter(c) for c in self.chaps]
        self.chapLengths = [len(c) for c in self.chapObjs]
        
        self.stats = {"filename":self.filename,
                      "textLength": self.length,
                      "numChapters": self.numChapters,
                      "chapterLengths": self.chapLengths}
        
    def findNamedAnchors(self, filename): 
        """ Find <a name=''> in HTML """
        p = re.compile('a name="(.*?)"', re.IGNORECASE)
        matches = p.finditer(self.rawText)
        return [m for m in matches]
    
    def naIsChapter(self, match):
        """ Is a named anchor a chapter heading? """
        aNameVal = match.groups()[0]
        pat = re.compile("^(c|ch|chap|2hch|2h_4_|link2HCH)\d+|CHAPTER_|^\d+$", re.IGNORECASE)
        test = pat.match(aNameVal)
        if test == None: 
            test = self.isRomanNumeral(aNameVal)
#         print(test)
        return True if test is not None else False
    
    def isRomanNumeral(self, string): 
        return re.match('^(?=[MDCLXVI])M{0,3}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$', string)
        
    def findH3(self):
        """ Find <h3 class='chapnum' in HTML. """
        p = re.compile('<h3 class="chapnum"', re.IGNORECASE)
        matches = p.finditer(self.rawText)
        return [m for m in matches]
    
    def getEndLocation(self):
        """
        Tries to find where the book ends.
        """
        ends = ["End of the Project Gutenberg",
                "End of Project Gutenberg's",
                "END OF THE PROJECT GUTENBERG EBOOK",
                "END OF THIS PROJECT GUTENBERG EBOOK"]
        for end in ends: 
            if end in self.rawText: 
                endLoc = self.rawText.find(end)
                if endLoc != -1: 
                    return endLoc
        return self.length
    
    def getTextBetweenHeadings(self):
        chapters = []
        lastHeading = len(self.chapLocs) - 1
        for i, headingLocation in enumerate(self.chapLocs):
            if i is not lastHeading:
                nextHeadingLocation = self.chapLocs[i+1]
                chapters.append(self.rawText[headingLocation-1:nextHeadingLocation])
        return chapters

class Chapter():
    def __init__(self, html):
        self.soup = BeautifulSoup(html, 'html.parser')
        self.text = self.soup.get_text()
        self.rawParas = self.getParas()
        self.paras = list(filter(self.isPara, self.rawParas))
        self.paraTexts = [""]
        self.paraTexts = [p.get_text().strip().replace('\n', ' ') for p in self.paras]
    
    def __len__(self): 
        return len(self.text)
    
    def __str__(self): 
        return self.text
    
    def getParas(self): 
        """ Extract paragraphs from HTML """
        ps = self.soup.find_all('p')
        return ps
    
    def isPara(self, p): 
        """ A paragraph needs to be at least 50 chars and have at least one period. 
        Takes a soup p object. """
        text = p.get_text()
        if len(text) < 50 or "." not in text: 
            return False
        else: 
            return True
        

In [7]:
text = Text(htmlFilenames[29])

Using file:  /run/media/jon/SAMSUNG/gitenberg/Warlord-of-Mars_68/68-h/68-h.htm


In [ ]:
# Start over. 
for f in glob('chaps-*'):
    os.remove(f)

for i in range(len(htmlFilenames)):
    fn = htmlFilenames[i]
    try:
        t = Text(fn)
    except: 
        continue 
    
    with open('chaps-Stats.json', 'a') as statsFile: 
        json.dump(t.stats, statsFile)
        statsFile.write('\n')
    
    with open('chaps-firstParas.txt', 'a') as firstsFile: 
        for chap in t.chapObjs: 
            try: 
                firstPara = chap.paraTexts[0] + '\n'
                firstsFile.write(firstPara)
            except: 
                continue
    with open('chaps-lastParas.txt', 'a') as lastsFile:
        for chap in t.chapObjs: 
            try: 
                lastPara = chap.paraTexts[-1] + '\n'
                lastsFile.write(lastPara)
            except: 
                continue
    with open('chaps-midParas.txt', 'a') as midsFile:
        for chap in t.chapObjs: 
            try: 
                midParas = chap.paraTexts[1:-1]
                midParasSample = random.sample(midParas, 4) # Just grab 4 middle paragraphs
                midParas = ' // '.join(midParasSample) + '\n'
                midsFile.write(midParas)
            except: 
                continue
    

Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Indiscreet-Letter_15728/15728-h/15728-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Jupiter-Weapon_27588/27588-h/27588-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Wind_22590/22590-h/22590-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Blister-Jones_19041/19041-h/19041-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Cup-of-Trembling-and-Other-Stories_36625/36625-h/36625-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Deaves-Affair_31361/31361-h/31361-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Huntress_26283/26283-h/26283-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Doubloons-and-the-Girl_31528/31528-h/31528-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/December-Love_6616/6616-h/6616-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/In-the-Wilderness_4603/4603-h/4603-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Happy-Warrior_38325/38325-h/38325-h.htm
Using file:

Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Golden-Road_316/316-h/316-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Moran-of-the-Lady-Letty_321/321-h/321-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/A-Knight-of-the-Cumberland_324/324-h/324-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Pupil_1032/1032-h/1032-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Rose-O-the-River_1033/1033-h/1033-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Joe-Wilson-and-His-Mates_1036/1036-h/1036-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-New-Machiavelli_1047/1047-h/1047-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Ruling-Passion--tales-of-nature-and-human-nature_1048/1048-h/1048-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Sartor-Resartus--the-life-and-opinions-of-Herr-Teufelsdr-ckh_1051/1051-h/1051-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Step-by-Step--or-Tidy-s-Way-to-Freedom_1052/1052-h/1052-h.htm
Using file:  /run/media/jon

Using file:  /run/media/jon/SAMSUNG/gitenberg/Riders-of-the-Purple-Sage_1300/1300-h/1300-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Ball-at-Sceaux_1305/1305-h/1305-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Magic-Skin_1307/1307-h/1307-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Selected-Stories-of-Bret-Harte_1312/1312-h/1312-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Over-the-Sliprails_1313/1313-h/1313-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Malefactor_1314/1314-h/1314-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Elizabeth-and-Her-German-Garden_1327/1327-h/1327-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/A-Voyage-to-Arcturus_1329/1329-h/1329-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Story-of-Little-Black-Sambo-and-The-Story-of-Little-Black-Mingo_1330/1330-h/1330-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Peter-Pan-in-Kensington-Gardens_1332/1332-h/1332-h.htm
Using file:  /run/media/jon/SAMS

Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Jungle_140/140-h/140-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Mansfield-Park_141/141-h/141-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The--30-000-Bequest-and-Other-Stories_142/142-h/142-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Mayor-of-Casterbridge_143/143-h/143-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Voyage-Out_144/144-h/144-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Middlemarch_145/145-h/145-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/A-Little-Princess--13-Being-the-whole-story-of-Sara-Crewe-now-told-for-the-first-time_146/146-h/146-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Lost-Continent_149/149-h/149-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Wild-Justice_152/152-h/152-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Jude-the-Obscure_153/153-h/153-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Rise-of-Silas-Lapham_154/154-h/154-h.h

Using file:  /run/media/jon/SAMSUNG/gitenberg/Cast-Upon-the-Breakers_399/399-h/399-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Blix_401/401-h/401-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Penrod_402/402-h/402-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Soldiers-of-Fortune_403/403-h/403-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Reporter-Who-Made-Himself-King_407/407-h/407-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Hell-Fer-Sartain-and-Other-Stories_410/410-h/410-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-King-s-Jackal_411/411-h/411-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Magic-of-Oz_419/419-h/419-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Dorothy-and-the-Wizard-in-Oz_420/420-h/420-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Kidnapped_421/421-h/421-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Kidnapped_421/421-h/files/relative.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Romany-

...

Using file:  /run/media/jon/SAMSUNG/gitenberg/Old-Celtic-Romances_38041/38041-h/38041-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/A-Duel_38054/38054-h/38054-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Out-of-the-Air_38060/38060-h/38060-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/White-Fire_38061/38061-h/38061-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/In-Mr.-Knox-s-Country_38062/38062-h/38062-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Sun-s-Babies_38063/38063-h/38063-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Aw-Aw-Tam-Indian-NightsBeing-the-myths-and-legends-of-the-Pimas-of-Arizona_38064/38064-h/38064-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Northwest-_38069/38069-h/38069-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Norwegian-Fairy-Book_38070/38070-h/38070-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/An-Ambitious-Woman--A-Novel_38075/38075-h/38075-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/A-M

Using file:  /run/media/jon/SAMSUNG/gitenberg/Endurance-Testor-How-Clear-Grit-Won-the-Day_38305/38305-h/38305-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/A-Galahad-of-the-Creeks--The-Widow-Lamport_38306/38306-h/38306-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Great-Hikeor-The-Pride-of-the-Khaki-Troop_38307/38307-h/38307-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Wolves-of-God-and-Other-Fey-Stories_38310/38310-h/38310-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/What-Will-People-Say--A-Novel_38311/38311-h/38311-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Mansion_38312/38312-h/38312-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Storm-Boundor-A-Vacation-Among-the-Snow-Drifts_38314/38314-h/38314-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/By-Desert-Ways-to-Baghdad_38319/38319-h/38319-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Chevalier-d-Auriac_38323/38323-h/38323-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg

Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Story-of-Charles-Strange-Vol.-1--of-3-A-Novel_38623/38623-h/38623-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Story-of-Charles-Strange-Vol.-2--of-3-A-Novel_38624/38624-h/36824-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Story-of-Charles-Strange-Vol.-3--of-3-A-Novel_38625/38625-h/38625-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/HeimatlosTwo-stories-for-children-and-for-those-who-love-children_38626/38626-h/38626-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Delilah-of-the-Snows_38635/38635-h/38635-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Eldest-Son_38646/38646-h/38646-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Honour-of-the-Clintons_38647/38647-h/38647-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Great-Opera-StoriesTaken-from-Original-Sources-in-Old-German_38654/38654-h/38654-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Love-Among-the-LionsA-Matrimonial-Expe

Using file:  /run/media/jon/SAMSUNG/gitenberg/Vignettes-of-Manhattan--Outlines-in-Local-Color_38918/38918-h/38918-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Pine-Needles_38922/38922-h/38922-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/SteelThe-Diary-of-a-Furnace-Worker_38932/38932-h/38932-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Camp-Fire-Girls-Larks-and-Pranks--Or-The-House-of-the-Open-Door_38934/38934-h/38934-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Little-Colonel-at-Boarding-School_38939/38939-h/38939-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Four-Years-A-Scout-and-Spy_38948/38948-h/38948-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Charles-Auchester-Volume-1-of-2_38949/38949-h/38949-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Cardigan_38958/38958-h/38958-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/First-at-the-North-Pole--Or-Two-Boys-in-the-Arctic-Circle_38968/38968-h/38968-h.htm
Using file:  /run/media/

Using file:  /run/media/jon/SAMSUNG/gitenberg/The-New-Rector_39215/39215-h/39215-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Snowball_39216/39216-h/39216-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-King-s-Stratagem-and-Other-Stories_39217/39217-h/39217-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Everlasting-Arms_39218/39218-h/39218-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/The-Mardi-Gras-Mystery_39229/39229-h/39229-h.htm
Using file:  /run/media/jon/SAMSUNG/gitenberg/Joel--A-Boy-of-Galilee_39231/39231-h/39231-h.htm


In [285]:
t.chapObjs[0].paraTexts[0]

'In the shadows of the forest that flanks the crimson plain by the\nside of the Lost Sea of Korus in the Valley Dor, beneath the hurtling\nmoons of Mars, speeding their meteoric way close above the bosom of\nthe dying planet, I crept stealthily along the trail of a shadowy\nform that hugged the darker places with a persistency that proclaimed\nthe sinister nature of its errand.'

In [296]:
[1,2,3,4,5][:-1]

[1, 2, 3, 4]